### Created by:- Rajat Chaudhari
### rajat.chaudhari@accenture.com
Reading analog gauge and detecting the gauge needle

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import imutils

In [2]:
cv2.__version__

'4.1.0'

In [3]:
cap=cv2.VideoCapture("../Gauge/data/straight_noOffset_notMoving.avi")

"""if cap.isOpened():
    ret, frame = cap.read()
else:
    ret=False"""
    
while True:
    ret, frame = cap.read()
    
    
    grey = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(grey)
    bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, -2)
    #(thresh, dst2) = cv2.threshold(grey, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    edges=cv2.Canny(bw,50,250,apertureSize=5,L2gradient=True)
    minLineLength = 10
    maxLineGap = 0
    lines = cv2.HoughLinesP(image=edges, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)
    #print(lines[0])
    for _,rho, theta,th in lines[0]:
        a=np.cos(theta)
        b=np.sin(theta)
        x0=a*rho
        y0=b*rho
        pts1=(int(x0 + 1000*(-b)),int(y0 + 1000*(a)))
        pts2=(int(x0 - 1000*(-b)),int(y0 - 1000*(a)))
        cv2.line(frame,pts1,pts2,(0,255,0),3)
    
    cv2.imshow("",frame)
    
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
cap.release()

In [3]:
cap=cv2.VideoCapture("../Gauge/video.mp4")

"""if cap.isOpened():
    ret, frame = cap.read()
else:
    ret=False"""
    
while True:
    ret, frame = cap.read()
    
    ret, frame = cap.read()
    grey = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #**********1
    #edges=cv2.Canny(grey,50,150,apertureSize=5,L2gradient=True)
    #**********1
    
    #**********2
    #(thresh, dst2) = cv2.threshold(grey, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    #plt.imshow(dst2)
    #dst2 = cv2.medianBlur(dst2, 5)
    #dst = cv2.GaussianBlur(grey, (5, 5), 0)
    #dst2 = cv2.Canny(dst2, 50, 150)
    #**********2
    
    #**********3
    """gray = cv2.bitwise_not(grey)
    bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, -2)"""
    #**********3
    
    minLineLength = 127
    maxLineGap = 0
    circles = cv2.HoughCircles(grey, cv2.HOUGH_GRADIENT, 1, 80)
    
    
    
    #*****************************************************************************************************
    a, b, c = circles.shape
    x,y,r = avg_circles(circles, b) 
    
    """x=344 #hard coding this for testing
    y=273
    r=127"""
    
    #draw center and circle
    cv2.circle(frame, (x, y), r, (0, 0, 255), 3, cv2.LINE_AA)  # draw circle
    cv2.circle(frame, (x, y), 2, (0, 255, 0), 3, cv2.LINE_AA)  # draw center of circle   
    cv2.imshow("",frame)
    
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
cap.release()

NameError: name 'avg_circles' is not defined

# Fitting this to our use case

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import imutils

In [3]:
def dist_2_pts(x1, y1, x2, y2):
    #print np.sqrt((x2-x1)^2+(y2-y1)^2)
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [4]:
def avg_circles(circles, b):
    avg_x=0
    avg_y=0
    avg_r=0
    for i in range(b):
        #optional - average for multiple circles (can happen when a gauge is at a slight angle)
        avg_x = avg_x + circles[0][i][0]
        avg_y = avg_y + circles[0][i][1]
        avg_r = avg_r + circles[0][i][2]
    avg_x = int(avg_x/(b))
    avg_y = int(avg_y/(b))
    avg_r = int(avg_r/(b))
    return avg_x, avg_y, avg_r

In [16]:
import time
millis = int(round(time.time() * 1000))
cap=cv2.VideoCapture("../Gauge/Video1.wmv")
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
if cap.isOpened():
    ret, frame = cap.read()
else:
    ret=False
  
while ret==True:
    try:
        ret, frame = cap.read()
        grey = cv2.cvtColor(frame.copy(),cv2.COLOR_BGR2GRAY)
        
        #**********1
        #edges=cv2.Canny(grey,50,150,apertureSize=5,L2gradient=True)
        #**********1

        #**********2
        #(thresh, dst2) = cv2.threshold(grey, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        #plt.imshow(dst2)
        #dst2 = cv2.medianBlur(dst2, 5)
        #dst = cv2.GaussianBlur(grey, (5, 5), 0)
        #dst2 = cv2.Canny(dst2, 50, 150)
        #**********2

        #**********3
        """gray = cv2.bitwise_not(grey)
        bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, -2)"""
        #**********3
        height, width = frame.shape[:2]
        minLineLength = 127
        maxLineGap = 0
        circles = cv2.HoughCircles(grey, cv2.HOUGH_GRADIENT, 1, 100)



        #*****************************************************************************************************
        a, b, c = circles.shape
        x,y,r = avg_circles(circles, b) 

        """x=344 #hard coding this for testing
        y=273
        r=127"""

        #draw center and circle
        cv2.circle(frame, (x, y), r, (0, 0, 255), 3, cv2.LINE_AA)  # draw circle
        cv2.circle(frame, (x, y), 2, (0, 255, 0), 3, cv2.LINE_AA)  # draw center of circle
        #plt.imshow(frame)
        cropSize = (r*2, r*2)

        icircle=circles[0,:][0]

        nx=x-int(icircle[0]-cropSize[1]//2)
        ny=y-int(icircle[1]-cropSize[0]//2)

        cropCoords = (max(0, icircle[1]-cropSize[0]//2),min(frame.shape[0], icircle[1]+cropSize[0]//2),
                      max(0, icircle[0]-cropSize[1]//2),min(frame.shape[1], icircle[0]+cropSize[1]//2)) 
        crop_cimg = frame[int(cropCoords[0]):int(cropCoords[1]),int(cropCoords[2]):int(cropCoords[3])]
        #plt.imshow(crop_cimg)
        #print(x,y,r)




        #for testing, output circles on image
        #cv2.imwrite('gauge-%s-circles.%s' % (gauge_number, file_type), img)


        #for calibration, plot lines from center going out at every 10 degrees and add marker
        #for i from 0 to 36 (every 10 deg)

        '''
        goes through the motion of a circle and sets x and y values based on the set separation spacing.  Also adds text to each
        line.  These lines and text labels serve as the reference point for the user to enter
        NOTE: by default this approach sets 0/360 to be the +x axis (if the image has a cartesian grid in the middle), the addition
        (i+9) in the text offset rotates the labels by 90 degrees so 0/360 is at the bottom (-y in cartesian).  So this assumes the
        gauge is aligned in the image, but it can be adjusted by changing the value of 9 to something else.
        '''
        separation = 10.0 #in degrees
        interval = int(360 / separation)
        p1 = np.zeros((interval,2))  #set empty arrays
        p2 = np.zeros((interval,2))
        p_text = np.zeros((interval,2))
        for i in range(0,interval):
            for j in range(0,2):
                if (j%2==0):
                    p1[i][j] = x + 0.9 * r * np.cos(separation * i * 3.14 / 180) #point for lines
                else:
                    p1[i][j] = y + 0.9 * r * np.sin(separation * i * 3.14 / 180)
        text_offset_x = 10
        text_offset_y = 5
        for i in range(0, interval):
            for j in range(0, 2):
                if (j % 2 == 0):
                    p2[i][j] = x + r * np.cos(separation * i * 3.14 / 180)
                    p_text[i][j] = x - text_offset_x + 1.2 * r * np.cos((separation) * (i+27) * 3.14 / 180) #point for text labels, i+9 rotates the labels by 90 degrees
                else:
                    p2[i][j] = y + r * np.sin(separation * i * 3.14 / 180)
                    p_text[i][j] = y + text_offset_y + 1.2* r * np.sin((separation) * (i+27) * 3.14 / 180)  # point for text labels, i+9 rotates the labels by 90 degrees

        #add the lines and labels to the image
        for i in range(0,interval):
            cv2.line(frame, (int(p1[i][0]), int(p1[i][1])), (int(p2[i][0]), int(p2[i][1])),(0, 255, 0), 2)
            cv2.putText(frame, '%s' %(int(i*separation)), (int(p_text[i][0]), int(p_text[i][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.3,(0,0,0),1,cv2.LINE_AA)    


        #*****************************************************************************************************
        min_angle = 0 #raw_input('Min angle (lowest possible angle of dial) - in degrees: ') #the lowest possible angle
        max_angle = 359 #raw_input('Max angle (highest possible angle) - in degrees: ') #highest possible angle
        min_value = 0 #raw_input('Min value: ') #usually zero
        max_value = 99 #raw_input('Max value: ') #maximum reading of the gauge
        units = 'mm' #raw_input('Enter units: ')
        thresh = 175
        maxValue = 255
        #plt.imshow(frame)
        #gray2 = cv2.cvtColor(crop_cimg, cv2.COLOR_BGR2GRAY)
        #plt.imshow(crop_cimg)
        #************1
        #gray = cv2.bitwise_not(grey)
        #bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, -2)
        #************1

        #************2
        (thresh, dst2) = cv2.threshold(grey, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        #************2

        #************3
        dst2 = cv2.medianBlur(dst2, 5)
        dst2 = cv2.GaussianBlur(dst2, (5, 5), 2)
        #dst2 = cv2.Canny(dst2, 50, 150)
        #************3

        #************4
        '''#dst2 = cv2.dilate(dst2, None, iterations=2)
        #cnts = cv2.findContours(dst2.copy(), cv2.RETR_EXTERNAL,
        #cv2.CHAIN_APPROX_SIMPLE)
        #cnts = imutils.grab_contours(cnts)'''
        #************4

        #************5
        fgmask = fgbg.apply(crop_cimg)
        #plt.imshow(fgmask)
        #plt.imshow(crop_cimg)
        #************5
        
        #************6
        fgmask = fgbg.apply(crop_cimg)
        dst3 = cv2.medianBlur(fgmask, 5)
        dst3 = cv2.GaussianBlur(dst3, (5, 5), 2)
        dst3 = cv2.Canny(dst3, 50, 150)
        #************6
        
        minLineLength = 30
        maxLineGap = 0
        #lines = cv2.HoughLinesP(image=gray2, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)
        #lines = cv2.HoughLinesP(image=bw, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)
        #lines = cv2.HoughLinesP(image=dst2, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)
        lines = cv2.HoughLinesP(image=fgmask, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)
        #print(lines[0])
        ##########################################################
        final_line_list = []
        #print "radius: %s" %r

        diff1LowerBound = 0.15 #diff1LowerBound and diff1UpperBound determine how close the line should be from the center
        diff1UpperBound = 0.25
        diff2LowerBound = 0.5 #diff2LowerBound and diff2UpperBound determine how close the other point of the line should be to the outside of the gauge
        diff2UpperBound = 1.0

        #print("length of lines %d"%len(lines))
        if lines is not None:
            for i in range(0, len(lines)):
                for x1, y1, x2, y2 in lines[i]:
                    diff1 = dist_2_pts(nx, ny, x1, y1)  # x, y is center of circle
                    diff2 = dist_2_pts(nx, ny, x2, y2)  # x, y is center of circle
                    #set diff1 to be the smaller (closest to the center) of the two), makes the math easier
                    if (diff1 > diff2):
                        temp = diff1
                        diff1 = diff2
                        diff2 = temp
                    # check if line is within an acceptable range
                    #if (((diff1<diff1UpperBound*r) and (diff1>diff1LowerBound*r) and (diff2<diff2UpperBound*r)) and (diff2>diff2LowerBound*r)):
                        line_length = dist_2_pts(x1, y1, x2, y2)
                        # add to final list
                        final_line_list.append([x1, y1, x2, y2])
                        #cv2.line(crop_cimg, (nx, ny), (x2, y2), (0, 255, 0), 2) #all lines
        if len(final_line_list)>0:
            x1 = final_line_list[0][0]
            y1 = final_line_list[0][1]
            x2 = final_line_list[0][2]
            y2 = final_line_list[0][3]
            
            ##########################################################
            #find the farthest point from the center to be what is used to determine the angle
            dist_pt_0 = dist_2_pts(nx, ny, x1, y1)
            dist_pt_1 = dist_2_pts(nx, ny, x2, y2)
            if (dist_pt_0 > dist_pt_1):
                x_angle = x1 - nx
                y_angle = ny - y1
                cv2.line(crop_cimg, (nx, ny), (x1, y1), (0, 255, 0), 2) #only 1 line
            else:
                x_angle = x2 - nx
                y_angle = ny - y2
                cv2.line(crop_cimg, (nx, ny), (x2, y2), (0, 255, 0), 2) #only 1 line

            print("final line",final_line_list[0])    
            print("x_angle , y_angle",x_angle, y_angle)

            # take the arc tan of y/x to find the angle
            #res = np.arctan2(np.divide(float(y_angle), float(x_angle)))
            res = (np.divide(float(y_angle), float(x_angle)))

            #np.rad2deg(res) #coverts to degrees
            print("arctan angle is %f" % ((res)))
            # print x_angle
            # print y_angle
            # print res
            # print np.rad2deg(res)

            #these were determined by trial and error
            res = np.rad2deg(abs(res))

            print("degrees %d" % ((res)))

            if x_angle > 0 and y_angle > 0:  #in quadrant I
                final_angle = res
            if x_angle < 0 and y_angle > 0:  #in quadrant II
                final_angle = 270+res
            if x_angle < 0 and y_angle < 0:  #in quadrant III
                final_angle = 180+res
            if x_angle > 0 and y_angle < 0:  #in quadrant IV
                final_angle = 90+res

            #print final_angle

            old_min = float(min_angle)
            old_max = float(max_angle)

            new_min = float(min_value)
            new_max = float(max_value)

            old_value = final_angle

            old_range = (old_max - old_min)
            new_range = (new_max - new_min)
            new_value = (((old_value - old_min) * new_range) / old_range) + new_min

            #print(new_value)
            #cv2.putText(frame, new_value, (int(r/2 - 20),int(r/2)), cv2.FONT_HERSHEY_DUPLEX, 3, (255, 0, 0),2)
            cv2.putText(frame, str(new_value), (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
            cv2.putText(frame, str(str(x_angle) +"  "+ str(y_angle)), (0, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
            cv2.putText(frame, str(final_angle), (0, 110), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
            cv2.imwrite('../Gauge/processed/VideoFunction/frame%s-lines.%s' % (millis, 'jpg'), frame)
            cv2.imshow("",frame)
        
        print(a)
    except:
        pass
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
cap.release()

final line []
x_angle , y_angle -108 51
arctan angle is -0.472222
degrees 27
1
final line []
x_angle , y_angle -127 70
arctan angle is -0.551181
degrees 31
1
final line []
x_angle , y_angle -112 55
arctan angle is -0.491071
degrees 28
1
final line []
x_angle , y_angle -113 56
arctan angle is -0.495575
degrees 28
1
final line []
x_angle , y_angle -108 51
arctan angle is -0.472222
degrees 27
1
final line []
x_angle , y_angle -116 59
arctan angle is -0.508621
degrees 29
1
final line []
x_angle , y_angle -109 52
arctan angle is -0.477064
degrees 27
1
final line []
x_angle , y_angle -113 56
arctan angle is -0.495575
degrees 28
1
final line []
x_angle , y_angle -116 59
arctan angle is -0.508621
degrees 29
1
final line []
x_angle , y_angle -113 56
arctan angle is -0.495575
degrees 28
1
final line []
x_angle , y_angle -108 51
arctan angle is -0.472222
degrees 27
1
final line []
x_angle , y_angle -107 50
arctan angle is -0.467290
degrees 26
1
final line []
x_angle , y_angle -129 72
arctan angle

In [6]:
crop_cimg

array([], shape=(0, 0, 3), dtype=uint8)

# Part 2

In [2]:
cap=cv2.VideoCapture("../Gauge/video.mp4")

"""if cap.isOpened():
    ret, frame = cap.read()
else:
    ret=False"""
    
while True:
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([30,150,50])
    upper_red = np.array([255,255,180])
    
    mask = cv2.inRange(hsv, lower_red, upper_red)
    res = cv2.bitwise_and(frame,frame, mask= mask)

    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    
    k = cv2.waitKey(5) & 0xFF
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
cap.release()

In [19]:
np.rad2deg(1.89)

108.28902327972558